In [1]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

import sqlite3
import pickle

from sklearn.preprocessing import StandardScaler


In [2]:
# conn = sqlite3.connect('/Users/aaronlevi/Documents/sql_db/chords_list.db')
conn = sqlite3.connect('/Users/aaronlevi/Documents/sql_db/pick-a-tune.db')
cur = conn.cursor()

In [3]:
# dfAllSongs = pd.read_csv('playlist_df.csv')
# dfAllSongs.head()

dfAllSongs = pd.DataFrame(columns = ['Song', 'Artist', 'All Chords', 'Chords', 'Label', 'Link']) 
dfAllSongs 

,Song,Artist,All Chords,Chords,Label,Link


In [4]:
chordPath  = '/Users/aaronlevi/Documents/Insight2020/chordRec/ug_chords/'
chordFiles = [f for f in listdir(chordPath) if isfile(join(chordPath, f))]

# songs_db = pd.read_sql_query("SELECT Song FROM basic_info", conn)
# SELECT count(*) FROM foos WHERE bar = 'baz'

# for every song in the directory...
for iSong in range(0, len(chordFiles)):
    dfThisSong = pd.read_csv(chordPath+chordFiles[iSong])
    
    # ...check if it's already in the db, and add it to the db if not
#     if sum(songs_db['Song'].str.contains( str(dfThisSong['Song'][0]) )) is 0:
    dfAllSongs = pd.concat([dfAllSongs, dfThisSong], axis=0)

In [5]:
dfAllSongs = dfAllSongs.reset_index(drop=True)
print(len(dfAllSongs))
dfAllSongs.head()

2515


,Song,Artist,All Chords,Chords,Label,Link
0,better-days,strung-out,"['F', 'C', 'Dm', 'Bb', 'C/E', 'Gm', 'F/A', 'Am...","Gm,C,Am,Dm,F/A,Bb,C/E,F",advanced,https://tabs.ultimate-guitar.com/tab/strung-ou...
1,mary-had-a-little-lamb,paul-mccartney,"['E', 'E7', 'A', 'Abm7', 'F#m7', 'E', 'A', 'Ab...","C,D7,Abm,Bb,C7,F7,Abm7,B7,G#dim,Am7,Am,Eb,G#,F...",advanced,https://tabs.ultimate-guitar.com/tab/paul-mcca...
2,mad-world,gary-jules,"['Em', 'A', 'Em', 'A', 'Em', 'G', 'D', 'A', 'E...","Em,A,D,G",novice,https://tabs.ultimate-guitar.com/tab/gary-jule...
3,magic-ways,tatsuro-yamashita,"['DbM7', 'Ebm9', 'DbM7', 'Ebm9', 'GbM7/Ab', 'D...","DbM7,Fm7,B13,DbM9,Bbm7/Eb,GbM7,Db9,Ebm9,GbM7/A...",advanced,https://tabs.ultimate-guitar.com/tab/tatsuro-y...
4,rhiannon,fleetwood-mac,"['Am', 'F', 'Am', 'F', 'Am', 'F', 'C', 'F', 'C...","F,C,Am",novice,https://tabs.ultimate-guitar.com/tab/fleetwood...


In [7]:
# add these songs to the sql db
dfAllSongs.to_sql('basic_info', con=conn, if_exists='append')

//anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [8]:
# 1331 songs in batch 1, 1099 new ones in batch 2
# print(len(dfAllSongs))

# new_songs_db = pd.read_sql_query("SELECT Song FROM basic_info", conn)
# new_songs_db['Song']

In [28]:
# now do the same for the features table in the db
# start by getting spotify credentials
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

spotifyCred = pd.read_csv('spotifyCred.csv')
client_credentials_manager = SpotifyClientCredentials(client_id= spotifyCred['0'][0],
                                                     client_secret= spotifyCred['0'][1])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [29]:
# and set up the "standard chord list" for comparison later
# put together a list of various standard major & minor chords
sl1 = ['A', 'Bb', 'B', 'C', 'C#', 'Db', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'G#', 'Ab' ]
sl2 = [s + 'maj' for s in sl1]
sl3 = [s + 'm' for s in sl1]
sl4 = [s + 'min' for s in sl1]

standardList = sl1+sl2+sl3+sl4


In [57]:
# preallocate 
nUniqueChords   = []
nHardChords     = []
chordDensity    = []
difficultyLabel = []

spotFeatures = {}

# spotFeatures['key'] = []
spotFeatures['tempo'] = []
# spotFeatures['duration_ms'] = []
spotFeatures['duration_m'] = []
spotFeatures['time_signature'] = []
spotFeatures['energy'] = []

# loop throiugh dfAllSongs and get spotify info
# this is only new songs
for iSong in range(0, len(dfAllSongs)):
    songName   = str(dfAllSongs['Song'][iSong]).replace('-', ' ').lower()
    artistName = str(dfAllSongs['Artist'][iSong]).replace('-', ' ').lower()

    # search spotify api
    result = sp.search(songName, limit=10) 
    nItems = len(result['tracks']['items'])

    # if our spotify search returns anything, extract desired vars
    if nItems>0:
        # match artist info, and get URI    
        foundMatch = [] # set up a counter...
        for iArtist in range(0, nItems):    
            if result['tracks']['items'][iArtist]['artists'][0]['name'].lower == artistName:
                songURI = result['tracks']['items'][iArtist]['uri']
                foundMatch.append(True)
            else:
                foundMatch.append(False) #... that keeps track of if we found a matching artist

        # if we didn't, let's assume the first artist was the correct one, and this was a 
        # problem with spelling/special chars/case sensitivity...
        if sum(foundMatch)==0:
            songURI = result['tracks']['items'][0]['uri']

        # pull audio features for a songURI
        features = sp.audio_features(songURI)

    #     spotFeatures['key'].append(features[0]['key'])
        spotFeatures['tempo'].append(features[0]['tempo'])
        spotFeatures['duration_m'].append((features[0]['duration_ms']/1000)/60)
        spotFeatures['time_signature'].append(features[0]['time_signature'])
        spotFeatures['energy'].append(features[0]['energy'])

    # if not, fill with NaNs
    else:
        spotFeatures['tempo'].append(float('nan'))
        spotFeatures['duration_m'].append(float('nan'))
        spotFeatures['time_signature'].append(float('nan'))
        spotFeatures['energy'].append(float('nan'))
        
    # get the chord information
    uniqChords = dfAllSongs['Chords'][iSong].split(',')
    nUniqueChords.append(len(uniqChords))    
#     chordDensity.append(nUniqueChords[iSong]/spotFeatures['duration_m'][iSong])
    
    nTotalChanges = len(dfAllSongs['All Chords'][iSong].split(','))
    chordDensity.append(nTotalChanges/spotFeatures['duration_m'][iSong])

    tmpHard = []
    for kCh in range(0, nUniqueChords[iSong]):
        tmpHard.append(uniqChords[kCh] not in standardList)

    nHardChords.append(sum(tmpHard))


In [59]:
# combine everything into one DF called allFeatures
spotFeatures = pd.DataFrame.from_dict(spotFeatures)
# spotFeatures.head()
allFeatures = spotFeatures

allFeatures['n_unique_chords']     = nUniqueChords
allFeatures['n_difficult_chords']  = nHardChords
allFeatures['chord_per_min']       = chordDensity

print(len(allFeatures))
allFeatures.head()

2515


,tempo,duration_m,time_signature,energy,n_unique_chords,n_difficult_chords,chord_per_min
0,89.132,2.408833,4.0,0.7260,8,2,47.740953
1,76.364,1.981967,4.0,0.0439,21,11,56.509528
2,172.286,3.533967,4.0,0.2920,4,0,17.826993
3,96.996,1.981967,4.0,0.7710,13,13,43.895794
4,129.012,4.212883,4.0,0.4900,3,0,13.292559


In [60]:
# append to the sql db
allFeatures.to_sql('features', con=conn, if_exists='append')


In [30]:
# cur.close()
# conn.close()

In [65]:
# find nans
index = allFeat['tempo'].index[allFeat['tempo'].apply(np.isnan)]
print(index)

Int64Index([181, 195, 482, 496, 1032, 1318, 1458, 1603, 2256, 2281, 2359,
            2435],
           dtype='int64')


In [82]:
# load the model
# filename = 'lr_adv_model_0930.sav'
filename = 'model_files/lr_adv_model_1004.sav'
lr_adv = pickle.load(open(filename, 'rb'))

In [83]:
allFeat = pd.read_sql_query("SELECT * FROM features", conn) #get from sql
# allFeat.dropna(axis=0, how='any', inplace=True) # remove nans
allFeat.reset_index(drop=True, inplace=True) #reset index
allFeat.drop('index', axis=1, inplace=True) # drop the extra column sql added

scaler = StandardScaler()

scaler.fit(allFeat)
feat_scale = scaler.transform(allFeat)

In [84]:
adv_predictions = lr_adv.predict(feat_scale)
adv_predictions

array([0., 1., 0., ..., 1., 0., 1.])

In [85]:
# load the model
filename = 'model_files/lr_nov_model_1004.sav'
lr_nov = pickle.load(open(filename, 'rb'))

In [86]:
nov_predictions = lr_nov.predict(feat_scale)
nov_predictions

array([1., 0., 1., ..., 0., 1., 0.])

In [87]:
# get ult-guitar labels from basic info table
label = pd.read_sql_query("SELECT Label FROM basic_info", conn)

# add columns for the two model predictions
label['model_isAdv']= adv_predictions
label['model_isNov']= nov_predictions

# rename first column
label.rename(columns={'Label': 'ug_label'})

label.head()

,Label,model_isAdv,model_isNov
0,advanced,0.0,1.0
1,advanced,1.0,0.0
2,novice,0.0,1.0
3,advanced,1.0,0.0
4,novice,0.0,1.0


In [92]:
# save a new table in the sql db called 'label'
label.to_sql('label', con=conn, if_exists='append')

In [88]:
dummyLabel=pd.get_dummies(label['Label'])
dummyLabel.head()

,advanced,intermediate,novice
0,1,0,0
1,1,0,0
2,0,0,1
3,1,0,0
4,0,0,1


In [90]:
print( np.mean(dummyLabel['advanced']==label['model_isAdv']) )
print( np.mean(dummyLabel['novice']==label['model_isNov']) )

0.7778665601278466
0.6352377147423093
